# Analiza okoljskih in zdravstvenih podatkov

In [1]:
"""Module providingFunction printing
python version."""
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind


pd.options.mode.chained_assignment = \
    None

### Opening the dataset
Read data from Excel file

In [2]:
df311 = pd.read_excel('Podatki_Podravje_temp_postaja_311.xlsx')
df311['datum'] = pd.to_datetime(df311['datum'])
df311 = df311[~(df311['datum'] < '2012-01-01')]
df311 = df311.reset_index(drop=True)
display(df311)

,postaja,datum,t7,t14,t21,tpov,tmax,tmin
0,311,2012-01-01,-0.6,4.5,0.3,1.1,5.7,-1.3
1,311,2012-01-02,-1.6,11.0,9.3,7.0,11.4,-2.3
2,311,2012-01-03,5.8,8.0,5.4,6.2,10.5,3.2
3,311,2012-01-04,0.4,9.2,2.6,3.7,10.2,0.0
4,311,2012-01-05,2.9,7.0,3.7,4.3,7.3,-1.9
...,...,...,...,...,...,...,...,...
3283,311,2020-12-27,-5.4,-0.3,-1.2,-2.0,-0.3,-5.8
3284,311,2020-12-28,0.9,2.3,1.4,1.5,3.0,-1.3
3285,311,2020-12-29,6.0,5.4,5.2,5.4,6.9,0.6
3286,311,2020-12-30,2.9,8.5,4.0,4.8,8.5,1.8


### Preparing the dataset 
Add a column with boolean values that determine whether there was a heat wave on a particular day.

In [3]:
df311['90th_centile'] = np.nan

for i, row in df311.iterrows():
    datum = row['datum']
    max_temp = row['tmax']

    zac_datum = datum - pd.Timedelta(
        days=15)
    kon_datum = datum + pd.Timedelta(
        days=15)
    temp_data = df311.loc[(df311[
                               'datum'] >= zac_datum) & (
                                  df311[
                                      'datum'] <= kon_datum), 'tmax']

    percentile = np.percentile(
        temp_data, 90)
    df311.at[
        i, '90th_centile'] = percentile

    p25 = np.percentile(temp_data, 25)
    df311.at[i, '25th_centile'] = p25
    p75 = np.percentile(temp_data, 75)
    df311.at[i, '75th_centile'] = p75

# Create boolean column for
# consecutive days above 90th percentile
pogoj = df311['tmax'] > df311[
    '90th_centile']
df311['dan_prek_90_centil'] = [
                                        False] * \
                                    df311.shape[
                                        0]
df311['dan_prek_90_centil'][
    pogoj] = True

# Create boolean column for heat waves
df311['vrocinski_val'] = df311[
                             'dan_prek_90_centil'] & \
                         df311[
                             'dan_prek_90_centil'].shift(
                             1) & \
                         df311[
                             'dan_prek_90_centil'].shift(
                             2)
df311['vrocinski_val'] |= df311[
                              'vrocinski_val'].shift(
    -1) | df311['vrocinski_val'].shift(
    -2)
df311['vrocinski_val'] = df311[
    'vrocinski_val'].fillna(False)

display(df311)

,postaja,datum,t7,t14,t21,tpov,tmax,tmin,90th_centile,25th_centile,75th_centile,dan_prek_90_centil,vrocinski_val
0,311,2012-01-01,-0.6,4.5,0.3,1.1,5.7,-1.3,10.35,7.125,9.500,False,False
1,311,2012-01-02,-1.6,11.0,9.3,7.0,11.4,-2.3,10.32,6.600,9.400,True,False
2,311,2012-01-03,5.8,8.0,5.4,6.2,10.5,3.2,10.29,6.775,9.350,True,False
3,311,2012-01-04,0.4,9.2,2.6,3.7,10.2,0.0,10.26,6.950,9.300,False,False
4,311,2012-01-05,2.9,7.0,3.7,4.3,7.3,-1.9,10.23,6.675,9.250,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,311,2020-12-27,-5.4,-0.3,-1.2,-2.0,-0.3,-5.8,8.92,2.975,6.525,False,False
3284,311,2020-12-28,0.9,2.3,1.4,1.5,3.0,-1.3,9.34,2.950,6.550,False,False
3285,311,2020-12-29,6.0,5.4,5.2,5.4,6.9,0.6,9.76,2.925,6.575,False,False
3286,311,2020-12-30,2.9,8.5,4.0,4.8,8.5,1.8,10.18,2.900,6.600,False,False


A function that aligns the text in the cells when saving the dataframe as an Excel file

In [4]:
def align_center(j):
    """
    Function to align text in the
    center for pandas DataFrame styles.

    Args:
        j (str): Style string.

    Returns:
        str: Style string with center
        alignment.
    """
    return ['text-align: center' for _
            in j]

Create an Excel sheet with dates and boolean values that tell whether there was a heat wave on that date

In [5]:
# Copy a subset of columns from df311
# DataFrame
df_vrocinski_val = df311[
    ['datum', 'vrocinski_val']].copy()
# Create a writer for Excel file
writer = pd.ExcelWriter(
    "vrocinski_val.xlsx",
    engine="xlsxwriter",
    date_format="dd/mm/yyyy",
    datetime_format="dd/mm/yyyy")
# Apply style to the df_vrocinski_val
# DataFrame and write it to Excel sheet
df_vrocinski_val.style.apply(align_center,
                        axis=0).to_excel(
    writer, sheet_name="vroc_val")
# Get the workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets["vroc_val"]
# Set column width for the worksheet
(max_row, max_col) = df_vrocinski_val.shape
worksheet.set_column(1, max_col, 20)
# Close the writer
writer.close()

excel_df = pd.read_excel('vrocinski_val.xlsx', index_col=0)
display(excel_df)

,datum,vrocinski_val
0,2012-01-01,False
1,2012-01-02,False
2,2012-01-03,False
3,2012-01-04,False
4,2012-01-05,False
...,...,...
3283,2020-12-27,False
3284,2020-12-28,False
3285,2020-12-29,False
3286,2020-12-30,False


Count the number of heat waves

In [6]:
# Count occurrences of True values in
# 'vrocinski_val' column
STEVEC = 0
for x in df311['vrocinski_val']:
    if x:
        STEVEC += 1

my_list = []

# Calculate values for 'my_list'
# based on 'tmax' column values and
# 'p25' and 'p75' variables
for i in range(df311.shape[0]):
    if df311.at[i, 'tmax'] > p25:
        my_list.append((df311.at[
                            i,
                            'tmax'] -
                        p25) / (
                               p75 -
                               p25))
    else:
        my_list.append(0)

### Opening and preparing the second dataset
Open the dataset with data about causes of deaths, dates of deaths, municipalities of deaths and activity status of the deceased.
To be compatible with previous dataframe we have to delete rows where dates don't align with the ones from the first dataframe and 
rename the columns

In [7]:
# Read data from an Excel file
df = pd.read_excel(
    'Podaki o umrlih_po vzroku, '
    'datumu smrti, obcini in statusu '
    'aktivnosti_2012-2021.xlsx',
    sheet_name="Tabela"
)

# Delete rows, where date is after 2021-01-01
df = df[df['Datum smrti'] < pd.Timestamp('2021-01-01')]

# Rename the columns
new_column_names = {
    "vzroksmrti -združene kategorije": "vzrok",
    "obcina prebivalisca": "obcina",
    "status aktivnosti": "aktivnost"
}
df = df.rename(columns=new_column_names)

display(df)

seznam_obcin = df['obcina'].dropna().unique().tolist()
seznam_obcin.sort()
seznam_vzrokov = df['vzrok'].dropna().unique().tolist()
seznam_vzrokov.sort()
seznam_aktivnosti = df['aktivnost'].dropna().unique().tolist()
seznam_aktivnosti.sort()

,Leto,Datum smrti,vzrok,obcina,aktivnost,število umrlih
0,2012,2012-01-01,Bolezni dihal,NaN,NaN,0
1,2012,2012-01-01,Bolezni kože in podkožja,NaN,NaN,0
2,2012,2012-01-01,Bolezni krvi in krvotvornih organov ter nekate...,NaN,NaN,0
3,2012,2012-01-01,Bolezni mišičnoskeletnega sistema in vezivnega...,NaN,NaN,0
4,2012,2012-01-01,Bolezni obtočil,Miklavž na Dravskem polju,Upokojenec,do 5
...,...,...,...,...,...,...
74759,2020,2020-12-31,"Nosečnost, porod in poporodno obdobje",NaN,NaN,0
74760,2020,2020-12-31,"Prirojene malformacije, deformacije in kromoso...",NaN,NaN,0
74761,2020,2020-12-31,"Simptomi, znaki ter nenormalni klinični in lab...",NaN,NaN,0
74762,2020,2020-12-31,Srčna kap,Rače - Fram,Upokojenec,do 5


### t-test function

In [8]:
def t_test(df, tip, kategorija, df_vrocinski_val):
    """
        Performs a t-test for two samples
        on the 'Count' column of the
        merged data frame for a given
        category.

        Args:
            df (pd.DataFrame):
            Data frame containing the
            data of occurred deaths.
            tip (str): type of category.
            kategorija (str): category.
            df_vrocinski_val (pd.DataFrame):
            Data frame containing the
            data of heat waves.

        Returns:
            tuple: A tuple containing the
            p-value and the total count
            of deaths for the category.

        Description:
            This function performs a
            t-test for two samples on the
            'Count' column of the merged
            data frame for a given
            category.
            It calculates the total count
            of deaths and performs a t-test.
        """

    df_kategorije = df[df[tip] == kategorija]
    tmp1 = df_kategorije.groupby(
        "Datum smrti").size().values
    df_kategorije = \
        df_kategorije.drop_duplicates(
            subset="Datum "
                   "smrti").assign(
            Count=tmp1)
    df_kategorije.loc[df[
                      'število ' \
                      'umrlih'] == 0,
    'Count'] = 0
    df_kategorije = df_kategorije.drop(
        columns=['Leto',"obcina", "vzrok", "aktivnost",
                 'število umrlih'])

    df_kategorije = df_kategorije.rename(
        columns={
            'Datum smrti': 'datum'})

    merged_df = pd.merge(df_vrocinski_val,
                         df_kategorije,
                         on='datum',
                         how='outer')
    merged_df["Count"] = merged_df[
        "Count"].fillna(0)
    stevilo_pojavitev_smrti = \
        merged_df['Count'].sum()

    df_je_vrocinski_val = merged_df[
        merged_df[
            'vrocinski_val'] == True]
    df_ni_vrocinski_val = merged_df[
        merged_df[
            'vrocinski_val'] == False]

    _, p_value = ttest_ind(
        df_je_vrocinski_val['Count'],
        df_ni_vrocinski_val['Count'])

    return p_value, \
        stevilo_pojavitev_smrti

### Analysis function

In [9]:
def analiza(df, tip, stolpec_s_kategorijami, df_vrocinski_val):
    """
        Performs a t-test for two samples on the 'Count' column
        of the merged data frame for all categories of type tip.

        Args:
            df (pd.DataFrame): Data frame containing the data
            of occurred deaths.
            tip (str): The name of the category.
            stolpec_s_kategorijami (list): A list of all
            categories of type tip.
            df_vrocinski_val (pd.DataFrame): Data frame containing
            the data of heat waves.

        Returns:
            None

        Description:
            This function performs a t-test for two samples on the
            'Count' column of the merged data frame for all
            categories of type tip. It calculates the p-value and
            the total count of deaths for each category in the list
            of categories. The results are stored in a new data
            frame and saved as an Excel file.
    """

    df_rezultat = pd.DataFrame(stolpec_s_kategorijami,
                             columns=[
                                 tip])

    for kategorija in stolpec_s_kategorijami:
        p_vrednost, stevilo_pojavitev_smrti = \
            t_test(df, tip, kategorija, df_vrocinski_val)

        df_rezultat.loc[df_rezultat[tip] == kategorija,\
            'p_vrednost'] = p_vrednost
        df_rezultat.loc[df_rezultat[tip] == kategorija,\
            'stevilo_pojavitev_smrti'] = \
            int(stevilo_pojavitev_smrti)

    with pd.ExcelWriter(
            tip+
            '.xlsx',
            engine='xlsxwriter',
            engine_kwargs={'options': {
                'strings_to_numbers':
                    True}}) as writer:
        df_rezultat.to_excel(
            writer)

### Performing the analysis
Call the analysis function for municipalities, causes of deaths and activity status of the deceased

In [10]:
tip = ["obcina", "vzrok", "aktivnost"]
seznam_seznamov = [seznam_obcin, seznam_vzrokov, seznam_aktivnosti]
slovar_seznamov = dict(zip(tip, seznam_seznamov))

for tip, seznam in slovar_seznamov.items():
    analiza(df, tip, seznam, df_vrocinski_val)
    
    
df_ob = pd.read_excel('obcina.xlsx', index_col=0)
display(df_ob)
df_vz = pd.read_excel('vzrok.xlsx', index_col=0)
display(df_vz)
df_ak = pd.read_excel('aktivnost.xlsx', index_col=0)
display(df_ak)

,obcina,p_vrednost,stevilo_pojavitev_smrti
0,Benedikt,0.747739,196
1,Cerkvenjak,0.915428,221
2,Cirkulane,0.685868,256
3,Destrnik,0.951216,243
4,Dornava,0.623292,246
5,Duplek,0.346990,582
6,Gorišnica,0.190742,340
7,Hajdina,0.346246,296
8,Hoče - Slivnica,0.086052,915
9,Juršinci,0.598515,227


,vzrok,p_vrednost,stevilo_pojavitev_smrti
0,Bolezni dihal,0.942322,1749
1,Bolezni kože in podkožja,0.568189,25
2,Bolezni krvi in krvotvornih organov ter nekate...,0.524701,31
3,Bolezni mišičnoskeletnega sistema in vezivnega...,0.627718,126
4,Bolezni obtočil,0.898549,8469
5,Bolezni prebavil,0.092746,1693
6,Bolezni sečil in spolovil,0.804080,425
7,Bolezni živčevja,0.522298,697
8,Duševne in vedenjske motnje,0.406019,569
9,"Endokrine, prehranske in presnovne bolezni",0.704255,547


,aktivnost,p_vrednost,stevilo_pojavitev_smrti
0,"Brezposeln, išče prvo zaposlitev",0.436942,48
1,"Brezposeln, že bil zaposlen",0.456139,726
2,Druga neaktivna oseba,0.906975,1051
3,Kmet / pomagajoči družinski član,0.029262,111
4,"Oseba, stara 14 let ali manj",0.480226,40
5,Prejemnik socialnih transferjev,0.678281,396
6,Samozaposlen,0.571781,138
7,Upokojenec,0.563681,23643
8,"Učenec, dijak",0.153811,21
9,Zaposlen,0.900619,1452
